In [1]:
import cv2 as cv
import numpy as np
import os
from cvzone.HandTrackingModule import HandDetector

In [29]:
folderpath="virtual paints/"
detector=HandDetector(detectionCon=0.85)
mypaints=os.listdir(folderpath)
overlaylist=[]
drawcolor=(0,0,255)
for img in mypaints:
    image=cv.imread(f'{folderpath}/{img}')
    overlaylist.append(image)
# print(len(overlaylist))
header=overlaylist[0]
cap=cv.VideoCapture(0)
cap.set(3,1280)
cap.set(4,700)
xp,yp=0,0

imagecanvas=np.zeros((720,1280,3),np.uint8)
while True:
    ret,frame=cap.read()
    frame[0:125,0:1280]=header
    hands,frame=detector.findHands(frame,draw=True,flipType=False)
    if hands:
        lmList = hands[0]['lmList']
        
        
        finger_tip_indices = [8, 12, 16, 20]
        
        
        fingers_up = [1 if lmList[i][1] < lmList[i - 1][1] else 0 for i in finger_tip_indices]
        
        if fingers_up[0] and fingers_up[1]:
            if lmList[8][1]<125:
                print('on top')
                if 200<lmList[8][0]<300:
                    header=overlaylist[0]
                    drawcolor=drawcolor
                elif 400<lmList[8][0]<500:
                    header=overlaylist[1]
                    drawcolor=(0,255,255)
                elif 600<lmList[8][0]<700:
                    header=overlaylist[2]
                    drawcolor=(255,0,0)
                elif 700<lmList[8][0]<800:
                    header=overlaylist[3]
                    drawcolor=(0,255,0)
                elif 800<lmList[8][0]<900:
                    header=overlaylist[4]
                    drawcolor=(0,0,0)
            cv.rectangle(frame,(lmList[8][0],lmList[8][1]+15),(lmList[12][0],lmList[12][1]+15),drawcolor,-1)
                
        else:
            cv.circle(frame,(lmList[8][0],lmList[8][1]),(5),drawcolor,-1)
            if drawcolor==(0,0,0):
                cv.line(imagecanvas,(xp,yp),(lmList[8][0],lmList[8][1]),drawcolor,55)
                cv.circle(frame,(lmList[8][0],lmList[8][1]),(5),drawcolor,-1)
                
                
            
            if xp==0 and yp==0:
                xp,yp=lmList[8][0],lmList[8][1]
            cv.line(imagecanvas,(xp,yp),(lmList[8][0],lmList[8][1]),drawcolor,2)
            xp,yp=lmList[8][0],lmList[8][1]
        
    grayframe=cv.cvtColor(imagecanvas,cv.COLOR_BGR2GRAY) # making colors brighter
    _,imgInv=cv.threshold(grayframe,50,255,cv.THRESH_BINARY_INV)
    imgInv=cv.cvtColor(imgInv,cv.COLOR_GRAY2BGR)
    frame=cv.bitwise_and(frame,imgInv)
    frame=cv.bitwise_or(frame,imagecanvas)
    
    
    frame=cv.addWeighted(frame,1.0,imagecanvas,1.0,0)
    
    cv.imshow('frame',frame)
    cv.imshow('canvas',imagecanvas)
    key=cv.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv.destroyAllWindows()
    